## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-19-14-53-42 +0000,bbc,Swiss deadly fire bar owners' lawyers condemn ...,https://www.bbc.com/news/articles/c5yvl4x3nqeo...
1,2026-01-19-14-52-09 +0000,nypost,Police probe if dead backpacker was ‘attacked’...,https://nypost.com/2026/01/19/world-news/polic...
2,2026-01-19-14-48-41 +0000,nyt,Live Updates: At Least 39 Are Dead in High-Spe...,https://www.nytimes.com/live/2026/01/18/world/...
3,2026-01-19-14-48-40 +0000,nyt,Survivors of Deadly Train Crash in Spain Descr...,https://www.nytimes.com/2026/01/19/world/europ...
4,2026-01-19-14-47-42 +0000,nyt,Splits Emerge Among Venezuelans as Revolutiona...,https://www.nytimes.com/2026/01/19/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,63
46,greenland,30
79,tariff,13
149,new,13
25,spain,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
252,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...,132
56,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...,130
243,2026-01-18-18-20-27 +0000,nypost,EU threatens trade ‘bazooka’ against US after ...,https://nypost.com/2026/01/18/world-news/europ...,128
221,2026-01-18-21-05-32 +0000,bbc,EU weighs response to Trump's tariff threat ov...,https://www.bbc.com/news/articles/c87r41j4n4jo...,126
260,2026-01-18-16-09-10 +0000,cbc,Canada 'concerned' about Trump's Greenland tar...,https://www.cbc.ca/news/politics/canada-greenl...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
252,132,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...
195,77,2026-01-19-00-00-06 +0000,nypost,At least 20 killed in high-speed train derailm...,https://nypost.com/2026/01/18/world-news/at-le...
250,44,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
34,41,2026-01-19-13-38-23 +0000,bbc,"Trade war in no-one's interest, says PM in fac...",https://www.bbc.com/news/articles/cvgjkrr4pk0o...
80,37,2026-01-19-11-25-10 +0000,nypost,Josh Shapiro says Harris team asked him if he ...,https://nypost.com/2026/01/19/us-news/josh-sha...
124,36,2026-01-19-09-16-33 +0000,nyt,European Union Officials Lean Toward Negotiati...,https://www.nytimes.com/2026/01/18/world/europ...
15,31,2026-01-19-14-22-01 +0000,nypost,Haunting image shows pile of abandoned shoes a...,https://nypost.com/2026/01/19/world-news/haunt...
230,30,2026-01-18-20-06-05 +0000,cbc,Trump's 'Board of Peace' proposal met with ske...,https://www.cbc.ca/news/world/gaza-ceasefire-p...
226,30,2026-01-18-20-13-23 +0000,nypost,Hundreds of thousands of french fries wash up ...,https://nypost.com/2026/01/18/world-news/hundr...
165,30,2026-01-19-03-40-25 +0000,nypost,Florida man randomly shot three tourists dead ...,https://nypost.com/2026/01/18/us-news/florida-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
